In [48]:
import tensorflow as tf
import numpy as np
import time
from functools import lru_cache
from typing import List, Tuple
from concurrent.futures import ThreadPoolExecutor


# Directions: up, down, left, right
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

In [42]:
def load_grid(file_path: str) -> np.ndarray:
    """
    Loads grid data from a text file into a 2D NumPy array.

    Parameters:
        file_path (str): Path to the input text file.

    Returns:
        np.ndarray: 2D array representing the grid.
    """
    grid = []
    with open(file_path, 'r') as file:
        for line_number, line in enumerate(file, start=1):
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            try:
                row = [int(char) for char in line]
            except ValueError as e:
                raise ValueError(f"Invalid character in line {line_number}: {e}")
            grid.append(row)
    return np.array(grid)

In [43]:
data = load_grid("test.txt")
data

array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [8, 7, 6, 5],
       [9, 8, 7, 6]])

In [30]:
def find_start_end_positions(grid: np.ndarray) -> Tuple[List[Tuple[int, int]], List[Tuple[int, int]]]:
    """
    Identifies all start (0) and end (9) positions in the grid.

    Parameters:
        grid (np.ndarray): 2D array representing the grid.

    Returns:
        Tuple[List[Tuple[int, int]], List[Tuple[int, int]]]:
            A tuple containing two lists:
                - List of (row, column) tuples for start positions.
                - List of (row, column) tuples for end positions.
    """
    start_positions = list(zip(*np.where(grid == 0)))
    end_positions = list(zip(*np.where(grid == 9)))
    return start_positions, end_positions

In [58]:
def find_all_trails_parallel(grid: np.ndarray) -> int:
    """
    Finds all valid trails from every 0 to 9 in the grid using parallel processing.

    Parameters:
        grid (np.ndarray): 2D array representing the grid.

    Returns:
        int: Total number of valid trails.
    """
    ROWS, COLS = grid.shape
    DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
    start_positions, end_positions = find_start_end_positions(grid)

    print(f"Start positions (0): {start_positions}")
    print(f"End positions (9): {end_positions}\n")

    def dfs(x: int, y: int, current_num: int) -> int:
        """
        Recursively counts the number of paths from (x, y) to any 9,
        where each move increases the number by exactly 1.

        Parameters:
            x (int): Current row.
            y (int): Current column.
            current_num (int): Current number on the grid.

        Returns:
            int: Number of valid paths to 9 from this cell.
        """
        # If current number is 9, we've reached the destination
        if current_num == 9:
            return 1

        total_paths = 0
        next_num = current_num + 1

        # Explore all four directions
        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            # Check boundaries
            if 0 <= nx < ROWS and 0 <= ny < COLS:
                if grid[nx][ny] == next_num:
                    total_paths += dfs(nx, ny, grid[nx][ny])

        return total_paths

    total_trails = 0
    with ThreadPoolExecutor() as executor:
        # Submit DFS tasks for all start positions
        futures = [executor.submit(dfs, x, y, grid[x][y]) for x, y in start_positions]

        # Aggregate the results correctly
        for idx, future in enumerate(futures, start=1):
            trails = future.result()
            print(f"Start {idx}: Position ({start_positions[idx-1][0]}, {start_positions[idx-1][1]}) with {grid[start_positions[idx-1][0], start_positions[idx-1][1]]} → {trails} trail(s)")
            total_trails += trails

    return total_trails


In [60]:
# Path to your input file
file_path = 'in.txt'  # Ensure this path is correct

# Load the grid
try:
    grid = load_grid(file_path)
except Exception as e:
    print(f"Error loading grid: {e}")


print("Loaded Grid:")
print(grid)
print("---------------------------------------------------")

# Find all valid trails
try:
    total_trails = find_all_trails_parallel(grid)
    print("---------------------------------------------------")
    print(f"Sum of all possible trails: {total_trails}")
except Exception as e:
    print(f"Error during DFS: {e}")

Loaded Grid:
[[7 8 9 ... 4 1 0]
 [8 7 6 ... 3 2 3]
 [9 0 5 ... 4 5 4]
 ...
 [8 9 4 ... 9 7 0]
 [3 4 5 ... 5 6 1]
 [4 5 6 ... 4 3 2]]
---------------------------------------------------
Start positions (0): [(0, 8), (0, 10), (0, 11), (0, 17), (0, 26), (0, 28), (0, 46), (0, 52), (0, 56), (1, 19), (1, 22), (1, 25), (1, 39), (2, 1), (2, 11), (2, 17), (2, 41), (2, 52), (3, 0), (3, 11), (3, 15), (3, 36), (3, 38), (3, 47), (4, 8), (4, 15), (4, 22), (4, 32), (4, 34), (4, 46), (4, 47), (5, 4), (5, 11), (5, 25), (5, 31), (5, 38), (5, 53), (6, 5), (6, 17), (6, 20), (6, 27), (6, 43), (6, 47), (6, 55), (7, 6), (7, 10), (7, 11), (7, 21), (7, 34), (7, 41), (7, 48), (8, 4), (9, 13), (9, 29), (9, 32), (9, 40), (10, 6), (10, 23), (10, 52), (11, 1), (11, 11), (11, 31), (11, 32), (11, 42), (11, 48), (11, 51), (11, 56), (12, 13), (12, 17), (12, 27), (12, 30), (13, 35), (13, 41), (14, 0), (14, 5), (14, 19), (14, 21), (14, 24), (14, 25), (14, 34), (14, 39), (15, 1), (15, 11), (15, 19), (15, 22), (15, 48), (1